# GTI771 - Apprentissage machine avancé
## Département de génie logiciel et des technologies de l’information



## Laboratoire 2 - Extraction de primitives
#### <font color=black> Version 2 - Janvier 2024 </font>

##### <font color=grey> Version 1 - Prof. Alessandro L. Koerich.
##### Version 2 - Chargé de lab. Arthur Josi

| NOMS                  | CODE PERMANENT                                   |
|-----------------------|--------------------------------------------------|
| Étudiant1             | Code1                                            |
| Étudiant2             | Code2                                            |
| Étudiant3             | Code3                                            |

## Introduction

Ce deuxième laboratoire porte sur la définition et l’extraction de primitives sur des visages. Vous aurez deux semaine pour compléter celui-ci.

De nouveau, le problème de classification qui vous est présenté est le problème [Facial Expression Recognition (FER)](https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge/data), dont le but est de classer des visages dans sept (7) catégories. En vous basant sur les concepts vus en classe et sur acquises en laboratoire, vous devez définir des primitives que vous jugez pertinentes à extraire sur ces types d’images et effectuer l’extraction de celles-ci sur l’ensemble de données fournies avec cet énoncé.

L’évaluation de ce laboratoire sera basée sur:
- la qualité des algorithmes proposés et utilisés;
- les réponses aux questions dans cette notebook;
- l'organisation de votre code source (SVP, n'oubliez pas de mettre des commentaires dans le code source!)

# Modules et bibliotèques python

### Import de bibliotèques

###  <font color=blue> À faire: </font>
1. Ajouter les bibliothèques que vous avez utilisées pour compléter ce notebook dans une cellule avec une petite description.

In [ ]:
import numpy as np  # package for scientific computing with Python.
import matplotlib.pyplot as plt # 2D plotting library

# Partie 1 - Ensemble de données

Point de départ: *fer2013-clean.csv* ou *fer2013-clean-pre.csv*

## 1a: Charger le fichier de données nettoyé et normalisé

###  <font color=blue> À faire: </font>
1. Reprenez votre ensemble de données nettoyé et repérez les trois partitions de données: apprentissage, validation et test.

In [ ]:
# Load data - Au choix:

# ferData = np.loadtxt( 'fer2013-clean.csv', delimiter=',', dtype=str )
ferData = np.loadtxt( 'content/fer2013-clean-pre.csv', delimiter=',', dtype=str )

training_data = ferData[ferData[:, 2] == 'Training']
validation_data = ferData[ferData[:, 2] == 'PublicTest']
test_data = ferData[ferData[:, 2] == 'PrivateTest']

def transform_str_float(d):
    return np.array([np.fromstring(row, sep=' ', dtype=float) for row in d])

# Training set
Xtrain = transform_str_float(training_data[:, 1])
ytrain = np.array(training_data[:,0], dtype=np.float32)

# Validation set
Xval = transform_str_float(validation_data[:, 1])
yval = np.array(validation_data[:,0], dtype=np.float32)

# # Test set
Xtest = transform_str_float(test_data[:, 1])
ytest = np.array(test_data[:,0], dtype=np.float32)

print(Xtrain.shape, Xval.shape, Xtest.shape)

In [ ]:

Xtrain = Xtrain.reshape( Xtrain.shape[0], 1, 48, 48 ).astype('uint8')
Xtest  = Xtest.reshape( Xtest.shape[0], 1, 48, 48 ).astype('uint8')
Xval   = Xval.reshape( Xval.shape[0], 1, 48, 48 ).astype('uint8')

# Partie 2: Extraction de primitives

Vous devez faire une recherche bibliographique pour trouver quelles sont les primitives qui sont plus souvent utilisées pour la reconnaissance des expressions faciales, et pour mieux comprendre leur intérêt. Voici quelques sources et mots-clés pour guider votre recherche:

- http://www.inf.ufpr.br/lesoliveira/download/ESWA2013.pdf
- https://doi.org/10.1016/j.patrec.2015.06.007
- https://doi.org/10.1109/FG.2011.5771374
- https://www.hindawi.com/journals/ijbi/2015/267807/
- https://link.springer.com/article/10.1007/s00500-020-05550-y
- https://ieeexplore.ieee.org/document/9232510/
- https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=9136619
- https://ietresearch.onlinelibrary.wiley.com/doi/10.1049/iet-ipr.2018.6647


**Mots-clés**: feature extraction, facial expression recognition, facial emotion recognition.

Bibliothèques Python pour les algorithmes d'extraction de primitives :
* [Scikit-image](https://scikit-image.org/docs/dev/)
* [OpenCV](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_tutorials.html)
* [Scikit-learn](https://scikit-learn.org/stable/modules/feature_extraction.html)
* [BiT](https://pypi.org/project/Bitdesc/)
* [E-BiT](https://github.com/stevetmat/BioInspiredFDesc)

Exemples d'algorithmes d'extraction de primitive :

* LBP, LPQ, Gabor filters, SIFT, SURF, HOG, GLCM, Haralick Moments, BiT, E-BiT, etc.

## 2a: Choix d'un algorithme d'extraction des primitives

###  <font color=blue> À faire: </font>
1. Choisir un algorithme d'extraction de primitives
  - #### <font color=red> Attention! Les équipes doivent utiliser des primitives différentes! Il n'y a pas de mauvais choix, seulement des approaches différentes, pas d'inquiétude. </font>
  - Chaque équipe doit poster/publier un message dans le <font color=red>"forum TP2"</font> avec son choix
  - Première arrivée première servie!<br><br>
2. Généralement, les algorithmes d'extraction de primitives ont plusieurs étapes. Vous devez étudier l'algorithme et la bibliothèque choisie pour bien comprendre l'algorithme, les primitives qu'il produit, et la manière dont vous devrez coder celui-ci. Aucune réponse n'est attendue pour cette question.
3. Nommer l'algorithme choisi et expliquer brièvement celui-ci, ses hyperparamètres, et le vecteur de sortie (dimension, type de variable).

#### Réponses

#### Explication :
Les filtres de Gabor sont utilisés pour analyser la texture et les caractéristiques spatiales des images. Un filtre de Gabor est composé d'une fonction sinus avec une fonction gausienne.

$G\left(x,y;\lambda,\theta,\psi,\sigma,\gamma \right) = exp\left(- {x'^2+\gamma^2y'^2 \over 2\sigma^2} \right) cos\left( 2π{x' \over \lambda } + \psi \right)$

où :
1. $x' = x \cos \theta + y \sin \theta$
2. $y' = - x \sin \theta + y \cos \theta$

#### Hyperparamètres :
Les principaux hyperparamètres des filtres de Gabor sont :

1. Lambda (𝜆) : La longueur d'onde de la sinusoïde, qui détermine la fréquence de la sinusoïde.
2. Theta (𝜃) : L'orientation de la normale au parallélogramme, qui définit l'orientation du filtre.
3. Psi (𝜓) : La phase de la fonction sinusoïdale.
4. Sigma (𝜎) : L'écart-type de la gaussienne, qui contrôle l'étendue du filtre.
5. Gamma (𝛾) : Le rapport d'aspect de la gaussienne, qui détermine l'élongation du filtre.

#### Vecteur de sortie : 
Le vecteur de sortie dépend des dimensions de l'image et le nombre de filtre appliqués. Dans notre cas où les images sont de tailles 48x48, le vecteur de sortie sera de 2304.Votre explication ici

## 2b: Extraction globale des primitives

###  <font color=blue> À faire: </font>
1. Extraire les primitives choisies des visages (ensembles d'apprentissage, validation et test). Un vecteur de primitives peut être, p. ex. 16 valeurs réelles représentant les sorties des filtres de Gabor, un ensemble de Haar-like features ou plusieurs valeurs réelles calculées par les descripteurs de Haralick (p. ex. contraste, homogénéité, etc.).
  - Chaque image de visage sera alors représentée par un vecteur $d-$dimensionnel.
  - Attention! Le résultat des algorithmes d'extraction de primitives doit être des vecteurs de primitives. Il y a des algorithmes qui nécessitent d'une étape supplémentaire comme calculer les histogrammes de primitives (p. ex. LBP, LPQ, SIFT, etc.)<br><br>

2. Sauvegarder les vecteurs de primitives obtenus sous la forme d'un fichier 'csv' (*fer2013-clean-lbp.csv* ou *fer2013-clean-pre-lbp.csv*). N'oubliez pas d'utiliser toujours la même structure que le fichier original *fer2013.csv*. Vous devez nommer vos fichiers de primitive en référence à la primitive utilisée, p. ex., *fer2013-clean-lbp.csv* pour LBP, *fer2013-clean-hog.csv* pour HOG, etc.

#### Code

In [ ]:
def mean_removal(X) -> np.ndarray:
    return X - np.mean(X, axis=0)

def normalize(X) -> np.ndarray:
    return X / 255



Xtrain_standardized = normalize(mean_removal(Xtrain))


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from joblib import Parallel, delayed
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler






def build_gabor_kernels(ksize=33, sigma=3.20, lambd=11.0, gamma=2.5, psi=-1, orientations=8, scales=5):
    kernels = []
    for scale in range(scales):
        for theta in np.linspace(0, np.pi, orientations, endpoint=False):
            kernel = cv2.getGaborKernel(
                (ksize, ksize), sigma +scale , theta, lambd, gamma, psi, ktype=cv2.CV_32F)
            kernels.append(kernel)
    return kernels

def apply_gabor_filters(image, kernels):
    filtered_images = []
    for kernel in kernels:
        filtered_image = cv2.filter2D(image, cv2.CV_8UC3, kernel)
        filtered_images.append(filtered_image)
    return filtered_images

def extract_features(filtered_images):
    features = []
    for img in filtered_images:
        mean = np.mean(img)
        variance = np.var(img)
        features.append(mean)
        features.append(variance)
        energy_val = np.sum(img**2)
        entropy_val = -np.sum(img * np.log(img + 1e-6))
        features.append(energy_val)
        features.append(entropy_val)
    return features

def load_data():
    # Placeholder function to load data
    # Replace this with actual data loading code
    #X = Xtrain # Dummy data
    X = Xtrain_standardized[0:10000]
    y = ytrain[0:10000]  # Dummy labels (binary classification)
    return X, y

def evaluate_parameters(params, X, y, orientations, scales):
    ksize, sigma, lambd, gamma, psi = params
    kernels = build_gabor_kernels(ksize=ksize, sigma=sigma, lambd=lambd, gamma=gamma, psi=psi, orientations=orientations, scales=scales)

    X_features = []
    for i in range(len(X)):
        image = X[i].reshape(48, 48)
        filtered_images = apply_gabor_filters(image, kernels)
        features = extract_features(filtered_images)
        X_features.append(features)

    X_features = np.array(X_features)
    X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.20, random_state=42)
    pipe = make_pipeline(StandardScaler(), SVC())
    pipe.fit(X_train, y_train)
    pipe.score(X_test, y_test)
    clf = SVC()
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Tested Params: ksize={ksize}, sigma={sigma}, lambd={lambd}, gamma={gamma}, psi={psi} => Accuracy: {accuracy * 100:.2f}%')

    return accuracy, params

def grid_search_gabor_params(X, y, ksize_values, sigma_values, lambd_values, gamma_values, psi_values, orientations=8, scales=5):
    best_accuracy = 0
    best_params = None

    param_sets = [(ksize, sigma, lambd, gamma, psi) 
                  for ksize in ksize_values 
                  for sigma in sigma_values 
                  for lambd in lambd_values 
                  for gamma in gamma_values 
                  for psi in psi_values]

    # Use joblib to evaluate the parameters in parallel
    results = Parallel(n_jobs=-1)(delayed(evaluate_parameters)(params, X, y, orientations, scales) for params in param_sets)
    
    print(f'Results: {results}')  # Debugging print
    
    # Find the best parameters based on the highest accuracy
    if results:
        best_accuracy, best_params = max(results, key=lambda x: x[0])
    
    print(f'Best Params: {best_params}, Best Accuracy: {best_accuracy}')  # Debugging print
    
    return best_params, best_accuracy


def visualize_kernels(kernels, orientations, scales):
    fig, axes = plt.subplots(scales, orientations, figsize=(12, 6))
    for i in range(scales):
        for j in range(orientations):
            kernel = kernels[i * orientations + j]
            ax = axes[i, j]
            ax.imshow(kernel, cmap='gray')
            ax.axis('off')
            ax.set_title(f'Scale {i+1}, Orientation {j+1}')
    plt.tight_layout()
    plt.show()

if __name__ == '__main__':
    # Load the dataset
    X, y = load_data()

    # Define the ranges for the Gabor parameters (coarse search)
    ksize_values = [31]
    sigma_values = [1.9]
    lambd_values = [6]
    gamma_values = [0.5,1]
    psi_values = [3,2.5]

    # Perform grid search
    best_params, best_accuracy = grid_search_gabor_params(X, y, ksize_values, sigma_values, lambd_values, gamma_values, psi_values, orientations=8, scales=5)

    print(f'Best Params: ksize={best_params[0]}, sigma={best_params[1]}, lambd={best_params[2]}, gamma={best_params[3]}, psi={best_params[4]}')
    print(f'Best Accuracy: {best_accuracy * 100:.2f}%')

    # Visualize an example image and its Gabor filtered responses with best parameters
    best_kernels = build_gabor_kernels(ksize=best_params[0], sigma=best_params[1], lambd=best_params[2], gamma=best_params[3], psi=best_params[4], orientations=8, scales=5)
    example_image = X[0].reshape(48, 48)
    filtered_images = apply_gabor_filters(example_image, best_kernels)

    visualize_kernels(best_kernels, orientations=8, scales=5)


In [ ]:

plt.figure(figsize=(12, 6))
plt.subplot(5, 8, 1)
plt.imshow(example_image, cmap='gray')
plt.title('Original Image')
plt.show()

visualize_kernels(filtered_images, orientations=8, scales=5)

plt.tight_layout()
plt.show()

# Visualize all the Gabor kernels
visualize_kernels(best_kernels, orientations=8, scales=5)

## 2c: Extraction locale des primitives

[Koutlas et Fotiadis (2008)](http://dx.doi.org/10.4018/9781605663142.ch016) ont proposé un ensemble de 20 points de référence. Selon les auteurs, ces points se situent autour des caractéristiques proéminentes du visage qui contiennent les informations les plus importantes concernant le mouvement musculaire responsable des expressions faciales.

Étant donné que vous ne connaissez pas la localisation de ces points de référence, une manière approximée d’extraire des informations locales des images de visage consiste à diviser l'image en $n$ petits zones de dimension égale  ($z_0, z_1,... ,z_n$) et d'extraire des primitives de chaque zone. Les primitives extraites de chaque zone sont ensuite concaténées en un seul vecteur de primitives.

![Exemples de FER](https://b2633864.smushcdn.com/2633864/wp-content/uploads/2021/04/face_reco_lbps_header.png?lossy=1&strip=1&webp=1)

###  <font color=blue> À faire: </font>
1. Utiliser une grille de dimension à votre choix (p. ex. $2$$\times$$2$, $4$$\times$$4$, $6$$\times$$6$, ...) de façon à créer de zones sur des images.
Remarque: La dimension de vos nouveaux ensembles de données pourrait maintenant être [samples][channels][nb_squares_per_img][width][height].
2. Répétez les pas de 2b, c.-à-d., éxtraire les primitives choisies de chaque cellule (zone) (ensembles d'apprentissage, validation et test). Un vecteur de primitives similaire à celui de la partie 2b sera produit pour chaque zone.
3. Concaténez séquentiellement les vecteurs générés pour chaque zone.

 - Chaque image de visage sera alors représentée par un vecteur de dimension $d$ $\times$ $nombre\_de\_zones$.<br><br>
4. Sauvegardez vos vecteurs de primitives sous la forme d'un fichier 'csv' (*fer2013-clean-lbp-zone4x4.csv* ou *fer2013-clean-pre-lbp-zone4x4.csv*). N'oubliez pas d'utiliser toujours la même structure du fichier original *fer2013.csv*. Vous devez nommer vos fichiers de primitive en référence à la primitive utilisée et la dimension de la grille choisie, p. ex., *fer2013-clean-lbp-zone4x4.csv* pour un zonage $4$$\times$$4$, *fer2013-clean-hog-zone8x8.csv* pour un zonage $8$$\times$$8$.

#### Code

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


# Get the dimensions of the image
height, width = masked_image.shape

# Define the number of rows and columns for the grid
rows = 4
cols = 4

# Calculate the size of each zone
zone_height = height // rows
zone_width = width // cols

# Create subplots for each zone
fig, axs = plt.subplots(rows, cols, figsize=(10, 10))

# Feature extraction function
def local_feature_extraction(image, rows=4, cols=4):
    height, width = image.shape
    zone_height = height // rows
    zone_width = width // cols

    feature = [[[] for _ in range(cols)] for _ in range(rows)]
    
    for i in range(rows):
        for j in range(cols):
            # Calculate the start and end indices for slicing
            start_row = i * zone_height
            end_row = (i + 1) * zone_height
            start_col = j * zone_width
            end_col = (j + 1) * zone_width
        
            # Slice the image to extract the current zone
            zone = image[start_row:end_row, start_col:end_col]
            
            # Calculate features for the zone
            mean = np.mean(zone)
            variance = np.var(zone)
            energy_val = np.sum(zone**2)
            entropy_val = -np.sum(zone * np.log(zone + 1e-6))
            
            # Append features to the list
            feature[i][j].append(mean)
            feature[i][j].append(variance)
            feature[i][j].append(energy_val)
            feature[i][j].append(entropy_val)
            
            # Optional: Display the zone
            axs[i, j].imshow(zone, cmap='gray')

    return feature

# Process each filtered image and collect features
info = []
for img in filtered_images:
    feature = local_feature_extraction(img)
    info.append(feature)

# Convert the list of features into a NumPy array with the desired structure
info_array = np.array(info)

# Reshape the array to have the desired dimensions: [num_images, rows, cols, extracted_features]
num_images = len(info)
rows, cols = len(info[0]), len(info[0][0])
extracted_features = len(info[0][0][0])  # Assuming all feature lists have the same length

info_array = info_array.reshape((num_images, rows, cols, extracted_features))

# To verify the structure
print(info_array.shape)


print(info_array.flatten().shape)


flattened_info_array = info_array.reshape((-1, extracted_features))

###NORMALISE
min_vals = np.min(flattened_info_array, axis=0)
max_vals = np.max(flattened_info_array, axis=0)
normalized_flattened_info_array = (flattened_info_array - min_vals) / (max_vals - min_vals)

# Reshape back to the original structure
normalized_info_array = normalized_flattened_info_array.reshape((num_images, rows, cols, extracted_features))
# Reshape back to the original structure
normalized_info_array = normalized_flattened_info_array.reshape((num_images, rows, cols, extracted_features))

# Convert to a more readable format
formatted_info_array = np.around(normalized_info_array, decimals=3)

# To verify the structure and values
print("Shape of formatted_info_array:", formatted_info_array.shape)
print("Formatted info array:\n", formatted_info_array)

# Check that values are within the range [0, 1]
print("Min value in formatted_info_array:", np.min(formatted_info_array))
print("Max value in formatted_info_array:", np.max(formatted_info_array))


# Partie 3: Normalisations des vecteurs de primitive
Avant d'entraîner un premier modèle d'apprentissage automatique avec les vecteurs de primitives (global et local) extrait dans la Partie 2, vous devez faire une normalisation des primitives.

La normalisation transforme les caractéristiques en échelles similaires (modifier la plage des valeurs) pour aider à améliorer les performances et la stabilité d'entraînement d'un modèle d’apprentissage machine. Notez que tous les ensembles de données n'ont pas nécessairement besoin d'être normalisés. Il n'est nécessaire que lorsque les plages d'attributs sont différentes.

Les algorithmes basés sur les mesures de distance telle que $k$-PPV et SVM sont les plus affectés par des différences dans les échelles. Les algorithmes d'apprentissage automatique qui utilisent l'algorithme descente de gradient dans l'entraînement tel que la régression linéaire et la régression logistique nécessitent une mise à l'échelle des données (même plage des valeurs) pour aider la descente de gradient à converger plus rapidement vers les minima. D'autre part, les algorithmes basés sur les arbres ne sont pas sensibles à l'échelle des caractéristiques. En effet, un arbre de décision ne divise qu'un nœud en fonction d'une seule primitive (élément du vecteur), et cette division n'est pas influencée par d'autres éléments (primitives) du vecteur.

###  <font color=blue> À faire: </font>
1. Vous pouvez retrouver des algorithmes de normalisation de la bibliothèque 'scikit-learn' [dans l'onglet preprocessing](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing)que vous avez par exemple discuté en cours et comprendre les différentes types de normalisation.
 - Voir aussi l'exemple [Compare the effect of different scalers on data with outliers](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#sphx-glr-auto-examples-preprocessing-plot-all-scaling-py)<br><br>
2. Choisir un algorithme pour normaliser vos vecteurs de primitives.
3. Normaliser vos vecteurs d'apprentissage, validation et test.

 - Attention! Utilisez *fit()* pour les données d'apprentissage et après *transform()* sur les trois partitions de données (apprentissage, validation et test).<br><br>
4. Sauvegardez vos vecteurs de primitives sous la forme d'un fichier 'csv' (*fer2013-clean-lbp-zone4x4-norm.csv* ou *fer2013-clean-pre-lbp-zone4x4-norm.csv*).


#### Code

In [ ]:
# Votre code ici

# Partie 4: Entraînement des modèle d'apprentissage machine

Vous êtes maintenant prêtes à entraîner un premier modèle d'apprentissage automatique avec les vecteurs de primitives (global et local) extraites dans la Partie 2.

## 4a: Entraîner un arbre de décision avec les primitives globales

###  <font color=blue> À faire: </font>
1. Étudiez l'algorithme [arbre de décision](https://scikit-learn.org/stable/modules/tree.html#tree-classification) de la bibliothèque `scikit-learn` pour bien comprendre les différents paramètres qui peuvent affecter l'entraînement, la généralisation et la complexité d’un modèle du type arbre de décision.
2. Étudiez les principaux hyperparamètres de [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier) qui peuvent aider à améliorer sa performance.
3. Choisir quelques hyperparamètres (au moins 2) et tourner un "grid search" sur l'ensemble de validation pour trouver les meilleures valeurs pour ces hyperparamètres.

- Astuce: Vous avez déjà un ensemble de validation! Alors, c'est préférable (beaucoup plus rapide!) d’utiliser la bibliothèque [hypopt](https://pypi.org/project/hypopt/) pour faire le réglage des hyperparamètres (grid seach)<br><br>

4. Une fois que vous avez trouvé le meilleur modèle, utiliser ces modèles pour faire la prédiction sur tous les exemples (apprentissage, validation, test) et rapporter les résultats (comme fait dans le TP1):<br>
4a. Rapport de classification produit avec *<font color=green>from sklearn.metrics import classification_report</font>*<br>
4b. taux de classification correct sur les trois (3) ensembles de données (sous la forme d'un tableau)<br>
4c. matrice de confusion produite avec *<font color=green> from sklearn.metrics import confusion_matrix</font>* pour les résultats sur l'ensemble de test (matrice 7 $\times$ 7 - étiquétte $\times$ prédictions)

5. Sauvegarder votre modèle dans un fichier *.pkl*. Regarder [model persistence](https://scikit-learn.org/stable/model_persistence.html)


#### Code

In [ ]:
# Votre code ici

## 4b: Entraîner un arbre de décision avec les primitives locales
###  <font color=blue> À faire: </font>
1. Refaire les étapes 3 à 5 de 4a)
2. Comparez les résultats avec ceux attendus avec les primitives globales et *template matching* (TP1)
3. Faire une brève analyse des résultats et présenter vos considérations et conclusions sur la performance de vos primitives + modèles.

#### Code

In [ ]:
# Votre code ici

#### Résultats et résponses

#### Vos répondes et résultats pour 4a) et 4b) ici:

##### Exemple:

Taux de classification (%)

| Ensemble | modèle TM   |  AD+LBP Global  | AD+LBP Local  |                                 
|----------|-------------|-----------------|---------------|
| App      | 99,67       |                 |               |                   
| Val      | 89,77       |                 |               |                             
| Test     | 77,99       |                 |               |        

# Fin